In [1]:
from pymoo.algorithms.moo.dnsga2 import DNSGA2
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.callback import CallbackCollection, Callback
from pymoo.core.problem import ElementwiseProblem, Problem
from pymoo.problems.functional import FunctionalProblem, func_return_none
from pymoo.optimize import minimize
from pymoo.problems.dyn import TimeSimulation
from pymoo.problems.dynamic.df import DF1
from pymoo.problems.multi import CTP3
from pymoo.problems.multi.srn import SRN
import matplotlib.pyplot as plt

import zuko
from zuko.flows import GMM
import numpy as np
import torch

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
panda = Robot(verbose=False)
# data generation
J, P = data_collection(robot=panda, N=config.num_train_size)
# build dimension reduction model
hnne = get_hnne_model(J, P)
# get loader
loader = get_loader(J, P, hnne=hnne)
# get val loader
J, P = data_collection(robot=panda, N=config.num_val_size)
val_loader = get_test_loader(J, P, F)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()

hnne load successfully from /home/luca/ikpflow/weights/panda/hnne.pickle
Model load successfully from /home/luca/ikpflow/weights/panda/nsf.pth


In [3]:
class ProblemWrapper(Problem):
    
    def __init__(self, 
                 end_effector_path,
                 model,
                 step_mod = 10,
                 l2_love_ratio = 0.3,
                 num_samples = 3,
                 **kwargs):
        self.end_effector_path = end_effector_path
        self.model = model
        self.step = 0
        self.step_mod = step_mod
        self.l2_love_ratio = l2_love_ratio
        self.constr_ieq = []
        self.num_samples = num_samples
        self.robot = Robot(verbose=False)
        self.solutions = np.zeros((len(end_effector_path), self.robot.dof))
        xl, xu = self.__x_bounds()
        super().__init__(n_var=len(xl),
                         n_obj=2,
                         n_ieq_constr=len(self.constr_ieq),
                         n_eq_constr=0,
                         xl=xl,
                         xu=xu,
                         **kwargs)
        
    def iksols(self, x):
        # need to check the output shapes for generating solutions
        pos = np.atleast_2d(self.pos()).repeat(repeats=len(x), axis=0)
        noise = np.zeros((len(pos),))
        x = torch.tensor(np.column_stack((pos, x, noise)), dtype=torch.float32).to(config.device)
        out = self.model(x).sample((self.num_samples,))
        out = out.detach().cpu().numpy()
        out = out.reshape((-1, self.robot.dof))
        return out
        
    def l2_err(self, iksols):
        errs = self.robot.l2_err_func_array(qs=iksols, ee_pos=self.pos())
        return errs
    
    def ang_jump(self, iksols):
        if self.index() == 0:
            return np.zeros((len(iksols), ))
        errs = (np.abs(iksols - self.solutions[self.index()-1])).sum(axis=1)
        return errs

    def __x_bounds(self):
        X_trans = load_numpy(file_path=config.x_trans_train_path)
        df = pd.DataFrame(data=X_trans)
        return df.min(axis=0).values, df.max(axis=0).values
    
    def pos(self):
        return self.end_effector_path[self.index()]
    
    def index(self):
        return self.step // self.step_mod
    
    def tic(self):
        self.step += 1
    
    def prune_samples(self, iksols, errs):
        ik_boxes = iksols.reshape((self.num_samples, -1, iksols.shape[-1]))
        err_boxes = errs.reshape((self.num_samples, -1, errs.shape[-1]))
        
        pop_size = ik_boxes.shape[1]
        survivors = np.zeros((pop_size, iksols.shape[-1]))
        surv_errs = np.zeros((pop_size, errs.shape[-1]))
        
        for pi in range(pop_size):
            survivors[pi], surv_errs[pi] = self.__get_best_iksols(iksols=ik_boxes[:, pi], errs=err_boxes[:, pi])
        return survivors, surv_errs
            
    def __get_best_iksols(self, iksols, errs):
        idx = np.argmin(self.love_index(errs))
        return iksols[idx], errs[idx]
    
    def love_index(self, errs):
        return errs[:, 0] * self.l2_love_ratio + errs[:, 1] * (1 - self.l2_love_ratio)
        
    def __set_best_iksols(self, iksols, errs):
        self.solutions[self.index()], _ = self.__get_best_iksols(iksols, errs)

    def _evaluate(self, x, out, *args, **kwargs):
        iksols = self.iksols(x)
        l2err = self.l2_err(iksols)
        ang_jump = self.ang_jump(iksols)
        errs = np.column_stack((l2err.reshape(-1, 1), ang_jump.reshape(-1, 1)))
        iksols, errs = self.prune_samples(iksols, errs)
        self.__set_best_iksols(iksols, errs)
        out["F"] = errs

In [4]:
traj_dir = sample_ee_traj(robot=panda, load_time='')
ee_traj = load_numpy(file_path=traj_dir + 'ee_traj.npy')
q_traj = load_numpy(file_path=traj_dir + 'q_traj.npy')

100%|██████████| 20/20 [00:00<00:00, 9729.31it/s]

mkdir /home/luca/ikpflow/data/panda/trajectory/06040017
/home/luca/ikpflow/data/panda/trajectory/06040017/ load successfully.


In [6]:
from pymoo.algorithms.moo.dnsga2 import DNSGA2
from pymoo.core.callback import CallbackCollection, Callback
from pymoo.optimize import minimize
from pymoo.problems.dyn import TimeSimulation
from pymoo.problems.dynamic.df import DF1
import matplotlib.pyplot as plt


problem = ProblemWrapper(end_effector_path=ee_traj, 
                         model=flow, 
                         step_mod=5,
                         l2_love_ratio=0.1)

algorithm = DNSGA2(pop_size=12)

simulation = TimeSimulation()

res = minimize(problem,
               algorithm,
               termination=('n_gen', len(ee_traj)*problem.step_mod),
               callback=CallbackCollection(simulation),
               seed=1,
               verbose=True)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       12 |      1 |             - |             -
     2 |       50 |      1 |  0.0007051798 |             f
     3 |       88 |      1 |  0.0003198579 |             f
     4 |      126 |      1 |  0.0003719057 |             f
     5 |      164 |      1 |  0.0006251257 |             f
     6 |      202 |      2 |  0.6110311631 |         ideal
     7 |      240 |      4 |  0.2471572853 |         ideal
     8 |      278 |      4 |  0.1996512952 |         ideal
     9 |      316 |      2 |  0.9967141883 |         ideal
    10 |      354 |      4 |  0.1372696591 |         ideal
    11 |      392 |      4 |  0.1195204744 |         ideal
    12 |      430 |      4 |  0.6667553931 |         ideal
    13 |      468 |      4 |  0.7065269341 |         ideal
    14 |      506 |      5 |  0.2192993260 |         ideal
    15 |      544 |      2 |  3.5082837126 |         ideal
    16 |      582 |      1 |  0.2379668951 |         ide

In [7]:
def cal_errs(robot = problem.robot, path = ee_traj, iksols = problem.solutions):
    errs = np.zeros((len(path),))
    ang_errs = np.zeros_like(errs)
    step = 0
    for q, ee_pos in zip(iksols, path):
        if step != 0:
            ang_errs[step] = np.sum(np.abs(iksols[step-1] - q))
        errs[step] = robot.l2_err_func(q=q, ee_pos=ee_pos)
        step += 1
    ang_errs[0] = np.average(ang_errs[1:])
    ang_errs = np.rad2deg(ang_errs)
    df = pd.DataFrame(np.column_stack((errs, ang_errs)), columns=['l2_err', 'ang_errs(sum)'])
    return df
df = cal_errs()
print(df.describe())

          l2_err  ang_errs(sum)
count  20.000000      20.000000
mean    0.002401      51.005169
std     0.001560      21.438395
min     0.000381      26.838667
25%     0.001235      35.293460
50%     0.002189      45.474734
75%     0.003379      62.757694
max     0.006619     107.072357
